In [1]:
!pip install transformers
!pip install comet_ml

     |████████████████████████████████| 2.0MB 14.8MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
     |████████████████████████████████| 3.2MB 49.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=78d7f35a0f5a5d200ad78e39b2a8e5f2eb8744f4939354cb376ab6919106721d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 266kB 16.6MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 522kB 49.7MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp37-none-any.whl size=34547 sha256=ebbc9a2b5449d49280e75bea33b7bba2e0a408551cb24f456cacbd655a70e94e
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [2]:
import comet_ml
from comet_ml import Experiment
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torch import optim
from transformers import BertForSequenceClassification
import pandas as pd
import numpy as np

In [3]:
from transformers import AutoTokenizer
class YelpDataset(Dataset):
    def __init__(self, file_path, pretrained_model):
        self.file_path = file_path 
        self.data = pd.read_json(file_path,lines=True)
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model,model_max_length=512)
        self.len = len(self.data)
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        label = row['stars']-1
        text = row['text']
        return {"text":text, "label":label}
def data_processing(data, tokenizer):
    text_batch = []
    label_batch = []
    for d in data:
        text_batch.append(d['text'])
        label_batch.append(d['label'])
    encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
    input_ids, attn_masks = encoding['input_ids'], encoding['attention_mask'].detach()
    labels = torch.tensor(label_batch).detach()
    return input_ids, attn_masks, labels

In [4]:
def val_err(model, validation_data, criterion):
    total_loss = 0
    count = 0
    for batch_idx, data in enumerate(validation_data):
        count += 1
        input_ids, attention_mask, labels = data
        outputs = model((input_ids, attention_mask))
        total_loss += criterion(outputs.logits, labels)
    return total_loss / count

In [6]:
def train(model, training_data, validation_data, optimizer, criterion, num_epoch, device, experiment, accumulation_steps):
    model.train()
    i = 0
    with experiment.train():
        for e in range(num_epoch):
            print('Epoch:', e)
            for batch_idx, data in enumerate(training_data):
                input_ids, attention_mask, labels = data
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                #optimizer.zero_grad()
                outputs = model((input_ids, attention_mask))
                loss = criterion(outputs.logits, labels)
                loss = loss/accumulation_steps  

                loss.backward() 
                if((i+1)%accumulation_steps)==0:
                    optimizer.step()       
                    optimizer.zero_grad()
                    print('Loss:',loss)
                experiment.log_metric('loss',loss.item(), step=i)
                #optimizer.step()
                i += 1

        print('validating...')
        print('Val Error: ', val_err(model, validation_data, criterion))

In [7]:
class YelpModel(nn.Module):
    def __init__(self, model_params):
        super(YelpModel, self).__init__()
        self.model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

    def forward(self, data):
        input_ids, attention_mask = data
        return self.model(input_ids, attention_mask = attention_mask, return_dict=True)

In [8]:
TRAIN_VAL_SPLIT = 0.8
hparams = {
    "batch_size":5,
    "learning_rate":1e-6,
    "num_epoch":1,
    "accumulation_steps":100
}
model_params = {}

In [9]:
import random
experiment = Experiment(api_key="mMEniBv0duELpAyET4ek2F3gF", project_name="cs182-nlp")
experiment.set_name(str(random.randint(0,1000)))

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/peter2000623/cs182-nlp/be2400c63fd34427848697f0783a19c5



In [10]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu') 
print(device)

cuda:0


In [11]:
dataset = YelpDataset("yelp_review_training_dataset.jsonl", "sentence-transformers/bert-base-nli-stsb-mean-tokens")
len_dataset = len(dataset)
train_dataset, val_dataset = random_split(dataset, [int(TRAIN_VAL_SPLIT*len_dataset), len_dataset-(int(TRAIN_VAL_SPLIT*len_dataset))])
train_dataloader = DataLoader(train_dataset, batch_size=hparams['batch_size'], collate_fn=lambda x: data_processing(x, dataset.tokenizer))
val_dataloader = DataLoader(val_dataset, batch_size=hparams['batch_size'], collate_fn=lambda x: data_processing(x, dataset.tokenizer))
print("data loaded...")

model = YelpModel(model_params).to(device)
optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
loss = nn.CrossEntropyLoss().to(device)

ValueError: ignored

In [12]:
print("training...")
train(model, train_dataloader, val_dataloader, optimizer, loss, hparams['num_epoch'], device, experiment, hparams['accumulation_steps'])

training...


NameError: ignored